In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objs as go

import plotly
plotly.offline.init_notebook_mode(connected=True)

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

In [ ]:
def load_audio(AUDIO_PATH):
    audio, sr = librosa.load(AUDIO_PATH)
    return audio, sr

def wav2melSpec(AUDIO_PATH):
    audio, sr = librosa.load(AUDIO_PATH)
    return librosa.feature.melspectrogram(y=audio, sr=sr)

def imgSpec(ms_feature):
    fig, ax = plt.subplots()
    ms_dB = librosa.power_to_db(ms_feature, ref=np.max)
    print(ms_feature.shape)
    img = librosa.display.specshow(ms_dB, x_axis='time', y_axis='mel', ax=ax)
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set(title='Mel-frequency spectrogram');

def hear_audio(AUDIO_PATH):
    audio, sr = librosa.load(AUDIO_PATH)
    print("\t", end="")
    ipd.display(ipd.Audio(data=audio, rate=sr))


def get_audio_info(path, show_melspec=False, label=None):
    spec = wav2melSpec(path)
    if label is not None:
        print("Label:", label)
    if show_melspec is not False:
        imgSpec(spec)
    hear_audio(path)

In [ ]:
class TextTransform:
    def __init__(self):
        self.char_to_num_dict =  {
      "а": 1, "б": 2, "в": 3, "г": 4, "д": 5, "е": 6, "ё": 7, "ж": 8, "з": 9, "и": 10, "й": 11,
      "к": 12, "л": 13, "м": 14, "н": 15, "о": 16, "п": 17, "р": 18, "с": 19, "т": 20, "у": 21,
      "ф": 22, "х": 23, "ц": 24, "ч": 25, "ш": 26, "щ": 27, "ь": 28, "ы": 29, "ъ": 30, "э": 31,
      "ю": 32, "я": 33, " ": 34}

        self.index_map = {}
        for key, value in self.char_to_num_dict.items():
            self.index_map[value] = key

    def char_to_num(self, sentence):
        num_sent = np.zeros(5000)
        i = 0
        for char in sentence.lower():
            if char in self.char_to_num_dict.keys():
                num_sent[i]=self.char_to_num_dict[char]
            i += 1
        return num_sent

    def num_to_char(self, sentence):
        char_sent = np.array()
        i = 0
        for num in sentence:
            char_sent[i]=self.index_map[num]
            i += 1
        return char_sent

In [ ]:
text_transform = TextTransform()

labels = []
for label in labels_uniq:
    labels.append(label)
    labels.append(label)

labels = list(map(text_transform.char_to_num, labels))

#MFCC coefficients



In [ ]:
import ConfigParser
import librosa
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from pprint import pprint

class cnnlib:
	def __init__(self):
		self.configParser = ConfigParser.RawConfigParser()
		self.configFilePath = r'../config/model.conf'
		self.configParser.read(self.configFilePath)
		self.channel = 1

		self.surah = int(self.configParser.get("ml-config","surah"))
		self.total_ayah = int(self.configParser.get("ml-config","total_ayah"))

		self.model = load_model("../generatedmodel/surah-"+str(self.surah)+"-model.h5",custom_objects={"f1": self.f1, "precision": self.precision})

	def wav2mfcc(self, file_path):
		max_pad_len=int(self.configParser.get("ml-config", "max_pad_len"))

		y, sr = librosa.load(file_path)
		y = librosa.effects.harmonic(y)
		mfcc = librosa.feature.tonnetz(y=y, sr=sr) #this is tonnetz, but i'm too lazy to change

		pad_width = max_pad_len - mfcc.shape[1]
		mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
		return mfcc

	# Metrics
	## https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
	def precision(self, y_true, y_pred):
        	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        	predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        	return true_positives / (predicted_positives + K.epsilon())

	def recall(self,y_true, y_pred):
        	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        	possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        	return true_positives / (possible_positives + K.epsilon())

	def f1(self, y_true, y_pred):
        	p = self.precision(y_true, y_pred)
        	r = self.recall(y_true, y_pred)
        	return 2*((p*r)/(p+r+K.epsilon()))

	def get_labels(self):
		labels = []
		for i in range(1, self.total_ayah+1):
			labels.append("ayat-"+str(i))

		label_indices = np.arange(0, len(labels))
		return labels, label_indices, to_categorical(label_indices)

	def isCorrect(self, location, label):
		print "Loading "+location+" ..."
		sample = self.wav2mfcc(location)
		sample_reshaped = sample.reshape(1, int(self.configParser.get("ml-config","shape_1")), int(self.configParser.get("ml-config","shape_2")), self.channel)
		answer = self.get_labels()[0][np.argmax(self.model.predict(sample_reshaped))]

		print("[DEBUG] Predicted label: "+answer+". Actual Label: ayat-"+label)
		if (answer == "ayat-"+label):
			return True
		else:
			return False

	def test(self):
		for i in range(1,self.total_ayah+1):
			sample = self.wav2mfcc("../testing/"+str(self.surah)+"/"+str(i)+".wav")
        		sample_reshaped = sample.reshape(1, int(self.configParser.get("ml-config","shape_1")), int(self.configParser.get("ml-config","shape_2")), self.channel)
			answer = self.get_labels()[0][np.argmax(self.model.predict(sample_reshaped))]
       			print("Predicted label: "+answer+". Actual Label: ayat-"+str(i))


#Train test split



In [ ]:

def cut_array(text_arr):
    return text_arr[:1756]

border = 3800

audio_train, text_train, audio_test, text_test = np.array(mfcc_spectograms[:3800]), np.array(list(map(cut_array, labels[:3800]))), np.array(mfcc_spectograms[3800:]), np.array(list(map(cut_array,labels[3800:])))
print(text_train.shape)

#Build a CNN



In [ ]:

dropout = 0.15
input_dim = (14, 2000,1)
output_dim_vocab = 34
output_dim = 1756



model = Sequential([
  layers.Conv2D(filters=32, kernel_size=[6, 6], strides=[2, 2],input_shape=input_dim, padding="same", use_bias=False),
  layers.BatchNormalization(),
  layers.Activation(activation='leaky_relu'),
  layers.MaxPooling2D(pool_size=(3,3)),
  layers.Dropout(dropout),
  # 2 layer
  layers.Conv2D(128,(3,3),padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation='relu'),
#   layers.MaxPooling2D(pool_size=(2,2)),
  layers.Dropout(dropout),
  # 3 conv layer
  layers.Conv2D(512,(3,3),padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation='relu'),
  layers.MaxPooling2D(pool_size=(2,2)),
  layers.Dropout(dropout),
  # 4 conv layer
  layers.Conv2D(512,(3,3),padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation='relu'),
#   layers.MaxPooling2D(pool_size=(2,2)),
  layers.Dropout(dropout),
  layers.Dense(512),
  layers.Activation(activation='leaky_relu'),
  layers.Dropout(dropout),
  layers.Conv2D(filters=32, kernel_size=[6, 6], strides=[1, 1], padding="same", use_bias=False),
#   fully cpnnected layer with 256 nuerons
  layers.Flatten(),
  layers.Dense(256, activation="softmax"),
  layers.BatchNormalization(),
  layers.Dropout(dropout),
  layers.Activation(activation='relu'),
#   fully cpnnected layer with 512 nuerons
  layers.Dense(output_dim_vocab, activation="softmax"),
  layers.Dense(512, activation="softmax"),
  layers.BatchNormalization(),
  layers.Dropout(dropout),
  layers.Activation(activation='relu'),
#     output layer
  layers.Dense(output_dim, activation="softmax"),
])

#CTC Loss Function



In [ ]:

def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0],dtype = 'int64')
    input_length = tf.cast(tf.shape(y_pred)[1],dtype = 'int64')
    label_length = tf.cast(tf.shape(y_true)[1],dtype = 'int64')

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype = 'int64')
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype = 'int64')
    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss='categorical_crossentropy',
#     loss=CTCLoss,
#     metrics=[CERMetric(), WERMetric()],
#     metrics=EditDistance())
    metrics=['accuracy'])

In [ ]:

# Define callbacks
model_path = 'Arabic_speech_model'

earlystopper = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min')
checkpoint = ModelCheckpoint(f"{model_path}/model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tb_callback = TensorBoard(f'{model_path}/logs', update_freq=1)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, min_delta=1e-10, patience=3, verbose=1, mode='auto')

In [ ]:
model.summary()

In [ ]:

# Train the model
history = model.fit(
    x=audio_train,
    y=text_train,
    validation_split=0.1,
    epochs=5, #Original is 50
    batch_size=26,
    callbacks=[earlystopper, checkpoint, reduceLROnPlat, tb_callback]
)

#History  Save to the next time since I could fix my error above.

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

# ________________ Graph 1 -------------------------

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

# ________________ Graph 2 -------------------------

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Categorical crossentropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.evaluate(audio_test, text_test)

In [ ]:

predict = model.predict(audio_test)
print(predict)